In [1]:
!pip install -r requirements.txt

In [2]:
# Load packages
import os
import pandas as pd
import numpy as np

In [3]:
%%time
df = pd.read_csv('data/ppp_loans.csv.gz', compression='gzip', low_memory=False)

CPU times: user 3min 22s, sys: 1min 34s, total: 4min 56s
Wall time: 5min 41s


In [4]:
df.head()

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit
0,0,5375617707,05/01/2020,101.0,PPP,NOT AVAILABLE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9551.0,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,NaN
1,1,9677497701,05/01/2020,464.0,PPP,NORTH CHARLESTON HOSPITALITY GROUP LLC,192 College Park Rd,Ladson,NaN,29456-3517,...,NaN,NaN,Limited Liability Company(LLC),19248.0,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN
2,2,9547167709,05/01/2020,464.0,PPP,Q AND J SERVICES LLC,301 Old Georgetown Road,Manning,NaN,29102-2734,...,NaN,NaN,Limited Liability Company(LLC),19248.0,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN
3,3,6448037706,05/01/2020,515.0,PPP,OPTIMIZED PROCESS SOLUTIONS DBA AAA INDUSTRIES,24500 CAPITOL,REDFORD,NaN,48239-2446,...,NaN,NaN,Limited Liability Company(LLC),9551.0,"Bank of America, National Association",CHARLOTTE,NC,Male Owned,Non-Veteran,NaN
4,4,9609017706,05/01/2020,464.0,PPP,"D2 POWER SPORTS, LLC",125 Simuel Dr.,Spartanburg,NaN,29303-2085,...,NaN,NaN,Limited Liability Company(LLC),19248.0,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN


In [5]:
#Verifying shape of data
df.shape

(10697219, 52)

In [6]:
%%time
#Dropping columns not needed
cols_to_drop = ['LoanNumber', 
                'SBAOfficeCode', 
                'ProcessingMethod', 
                'BorrowerName', 
                'BorrowerAddress', 
                'SBAGuarantyPercentage',
                'ServicingLenderLocationID', 
                'ServicingLenderAddress', 
                'ServicingLenderZip', 
                'ProjectZip', 
               'OriginatingLenderLocationID'
                ]

df = df.drop(cols_to_drop, axis=1)

CPU times: user 2.12 s, sys: 2.31 s, total: 4.43 s
Wall time: 4.87 s


In [7]:
#Verifying col drop
df.shape

(10697219, 41)

In [8]:
%%time
#Turn DateApproved and LoanStatusDate to Datetime
df['DateApproved'] = pd.to_datetime(df['DateApproved'])
df['LoanStatusDate'] = pd.to_datetime(df['LoanStatusDate'])

CPU times: user 4.44 s, sys: 2.12 s, total: 6.56 s
Wall time: 6.66 s


In [9]:
df.dtypes

Unnamed: 0                            int64
DateApproved                 datetime64[ns]
BorrowerCity                         object
BorrowerState                        object
BorrowerZip                          object
LoanStatusDate               datetime64[ns]
LoanStatus                           object
Term                                  int64
InitialApprovalAmount               float64
CurrentApprovalAmount               float64
UndisbursedAmount                   float64
FranchiseName                        object
ServicingLenderName                  object
ServicingLenderCity                  object
ServicingLenderState                 object
RuralUrbanIndicator                  object
HubzoneIndicator                     object
LMIIndicator                         object
BusinessAgeDescription               object
ProjectCity                          object
ProjectCountyName                    object
ProjectState                         object
CD                              

In [10]:
%%time
#Dict to store new value names  
rural_dict = {
    'R':'RURAL',
    'U':'URBAN'
}

hub_dict = {
    'Y':'YES',
    'N':'NO'
}

lmi_dict = {
    'Y':'YES',
    'N':'NO'
}

#Rename the values
df['RuralUrbanIndicator']=df['RuralUrbanIndicator'].replace(rural_dict)
df['HubzoneIndicator']=df['HubzoneIndicator'].replace(hub_dict)
df['LMIIndicator']=df['LMIIndicator'].replace(lmi_dict)

CPU times: user 3.33 s, sys: 207 ms, total: 3.54 s
Wall time: 3.6 s


In [11]:
#Verify new values
df['RuralUrbanIndicator']

0           URBAN
1           URBAN
2           URBAN
3           URBAN
4           URBAN
            ...  
10697214    URBAN
10697215    URBAN
10697216    URBAN
10697217    RURAL
10697218    RURAL
Name: RuralUrbanIndicator, Length: 10697219, dtype: object

In [12]:
#Verify new values
df['HubzoneIndicator']

0            NO
1            NO
2           YES
3           YES
4            NO
           ... 
10697214     NO
10697215     NO
10697216    YES
10697217     NO
10697218     NO
Name: HubzoneIndicator, Length: 10697219, dtype: object

In [13]:
#Verify new values
df['LMIIndicator']

0            NO
1            NO
2           YES
3            NO
4           YES
           ... 
10697214     NO
10697215     NO
10697216     NO
10697217     NO
10697218     NO
Name: LMIIndicator, Length: 10697219, dtype: object

In [14]:
%%time
#Dropping the duplicates
df = df.drop_duplicates()

CPU times: user 48.8 s, sys: 15.7 s, total: 1min 4s
Wall time: 1min 5s


In [15]:
#There are no duplicates
df.shape

(10697219, 41)

In [16]:
%%time
#Store columns in df
cols_to_uppercase = ['BorrowerCity',
                      'BorrowerState',
                      'LoanStatus',
                     'FranchiseName',
                     'ServicingLenderName',
                      'ServicingLenderCity',
                      'ServicingLenderState',
                       'RuralUrbanIndicator',
                      'HubzoneIndicator',
                      'LMIIndicator',
                     'BusinessAgeDescription',
                      'ProjectCity',
                      'ProjectCountyName',
                     'Race',
                     'Ethnicity',
                     'BusinessType',
                     'OriginatingLender',
                     'OriginatingLenderCity',
                     'OriginatingLenderState',
                     'Gender',
                     'Veteran',
                     'NonProfit']

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.25 µs


In [17]:
#Turn colums into string type
df[cols_to_uppercase] = df[cols_to_uppercase].astype(str)

In [18]:
#Verifying that type is str
type(df['BorrowerCity'][0])

str

In [19]:
%%time
#Uppercase strings in columns
df['BorrowerCity'] = df['BorrowerCity'].str.upper()
df['BorrowerState'] = df['BorrowerState'].str.upper()
df['LoanStatus'] = df['LoanStatus'].str.upper()
df['FranchiseName'] = df['FranchiseName'].str.upper()
df['ServicingLenderName'] = df['ServicingLenderName'].str.upper()
df['ServicingLenderCity'] = df['ServicingLenderCity'].str.upper()
df['ServicingLenderState'] = df['ServicingLenderState'].str.upper()
df['RuralUrbanIndicator'] = df['RuralUrbanIndicator'].str.upper()
df['HubzoneIndicator'] = df['HubzoneIndicator'].str.upper()
df['LMIIndicator'] = df['LMIIndicator'].str.upper()
df['BusinessAgeDescription'] = df['BusinessAgeDescription'].str.upper()
df['ProjectCity'] = df['ProjectCity'].str.upper()
df['ProjectCountyName'] = df['ProjectCountyName'].str.upper()
df['Race'] = df['Race'].str.upper()
df['Ethnicity'] = df['Ethnicity'].str.upper()
df['BusinessType'] = df['BusinessType'].str.upper()
df['OriginatingLender'] = df['OriginatingLender'].str.upper()
df['OriginatingLenderCity'] = df['OriginatingLenderCity'].str.upper()
df['OriginatingLenderState'] = df['OriginatingLenderState'].str.upper()
df['Gender'] = df['Gender'].str.upper()
df['Veteran'] = df['Veteran'].str.upper()
df['NonProfit'] = df['NonProfit'].str.upper()

CPU times: user 1min 31s, sys: 11.8 s, total: 1min 43s
Wall time: 1min 47s


In [20]:
#Verify the uppercase of values
df.head(5)

,Unnamed: 0,DateApproved,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,InitialApprovalAmount,CurrentApprovalAmount,...,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit
0,0,2020-05-01,NAN,NAN,NaN,NaT,EXEMPTION 4,24,148440.00,148440.00,...,NaN,NaN,NaN,NAN,"BANK OF AMERICA, NATIONAL ASSOCIATION",CHARLOTTE,NC,UNANSWERED,UNANSWERED,NAN
1,1,2020-05-01,LADSON,NAN,29456-3517,NaT,EXEMPTION 4,24,140081.78,140081.78,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),SYNOVUS BANK,COLUMBUS,GA,UNANSWERED,UNANSWERED,NAN
2,2,2020-05-01,MANNING,NAN,29102-2734,2021-04-20,PAID IN FULL,24,136520.96,136520.96,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),SYNOVUS BANK,COLUMBUS,GA,UNANSWERED,UNANSWERED,NAN
3,3,2020-05-01,REDFORD,NAN,48239-2446,2021-04-16,PAID IN FULL,24,126798.00,126798.00,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),"BANK OF AMERICA, NATIONAL ASSOCIATION",CHARLOTTE,NC,MALE OWNED,NON-VETERAN,NAN
4,4,2020-05-01,SPARTANBURG,NAN,29303-2085,NaT,EXEMPTION 4,24,125916.00,125916.00,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),SYNOVUS BANK,COLUMBUS,GA,UNANSWERED,UNANSWERED,NAN


In [21]:
%%time
#Turn str 'NAN' into NaN
df['BorrowerCity'] = df['BorrowerCity'].replace(['NAN'], np.nan)
df['BorrowerState'] = df['BorrowerState'].replace(['NAN'], np.nan)
df['LoanStatus'] = df['LoanStatus'].replace(['NAN'], np.nan)
df['FranchiseName'] = df['FranchiseName'].replace(['NAN'], np.nan)
df['ServicingLenderName'] = df['ServicingLenderName'].replace(['NAN'], np.nan)
df['ServicingLenderCity'] = df['ServicingLenderCity'].replace(['NAN'], np.nan)
df['ServicingLenderState'] = df['ServicingLenderState'].replace(['NAN'], np.nan)
df['RuralUrbanIndicator'] = df['RuralUrbanIndicator'].replace(['NAN'], np.nan)
df['HubzoneIndicator'] = df['HubzoneIndicator'].replace(['NAN'], np.nan)
df['LMIIndicator'] = df['LMIIndicator'].replace(['NAN'], np.nan)
df['BusinessAgeDescription'] = df['BusinessAgeDescription'].replace(['NAN'], np.nan)
df['ProjectCity'] = df['ProjectCity'].replace(['NAN'], np.nan)
df['ProjectCountyName'] = df['ProjectCountyName'].replace(['NAN'], np.nan)
df['Race'] = df['Race'].replace(['NAN'], np.nan)
df['Ethnicity'] = df['Ethnicity'].replace(['NAN'], np.nan)
df['BusinessType'] = df['BusinessType'].replace(['NAN'], np.nan)
df['OriginatingLender'] = df['OriginatingLender'].replace(['NAN'], np.nan)
df['OriginatingLenderCity'] = df['OriginatingLenderCity'].replace(['NAN'], np.nan)
df['OriginatingLenderState'] = df['OriginatingLenderState'].replace(['NAN'], np.nan)
df['Gender'] = df['Gender'].replace(['NAN'], np.nan)
df['Veteran'] = df['Veteran'].replace(['NAN'], np.nan)
df['NonProfit'] = df['NonProfit'].replace(['NAN'], np.nan)

CPU times: user 39.1 s, sys: 30.2 s, total: 1min 9s
Wall time: 1min 26s


In [22]:
#Verifying the replace to 
df.head(5)

,Unnamed: 0,DateApproved,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,InitialApprovalAmount,CurrentApprovalAmount,...,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit
0,0,2020-05-01,NaN,NaN,NaN,NaT,EXEMPTION 4,24,148440.00,148440.00,...,NaN,NaN,NaN,NaN,"BANK OF AMERICA, NATIONAL ASSOCIATION",CHARLOTTE,NC,UNANSWERED,UNANSWERED,NaN
1,1,2020-05-01,LADSON,NaN,29456-3517,NaT,EXEMPTION 4,24,140081.78,140081.78,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),SYNOVUS BANK,COLUMBUS,GA,UNANSWERED,UNANSWERED,NaN
2,2,2020-05-01,MANNING,NaN,29102-2734,2021-04-20,PAID IN FULL,24,136520.96,136520.96,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),SYNOVUS BANK,COLUMBUS,GA,UNANSWERED,UNANSWERED,NaN
3,3,2020-05-01,REDFORD,NaN,48239-2446,2021-04-16,PAID IN FULL,24,126798.00,126798.00,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),"BANK OF AMERICA, NATIONAL ASSOCIATION",CHARLOTTE,NC,MALE OWNED,NON-VETERAN,NaN
4,4,2020-05-01,SPARTANBURG,NaN,29303-2085,NaT,EXEMPTION 4,24,125916.00,125916.00,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),SYNOVUS BANK,COLUMBUS,GA,UNANSWERED,UNANSWERED,NaN


In [23]:
%%time
#Standarize Columns names 
df = df.rename(columns={'UTILITIES_PROCEED': 'UtilitiesProceed',
                        'PAYROLL_PROCEED':'PayrollProceed',
                        'MORTGAGE_INTEREST_PROCEED':'MortgageIntProceed',
                        'RENT_PROCEED':'RentProceed',
                        'REFINANCE_EIDL_PROCEED':'RefinanceEidlProceed',
                        'HEALTH_CARE_PROCEED':'HealthcareProceed',
                        'DEBT_INTEREST_PROCEED':'DebtIntProceed'
                       })

CPU times: user 5.44 s, sys: 27.9 s, total: 33.4 s
Wall time: 49 s


In [24]:
df.head(5)

,Unnamed: 0,DateApproved,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,InitialApprovalAmount,CurrentApprovalAmount,...,RefinanceEidlProceed,HealthcareProceed,DebtIntProceed,BusinessType,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit
0,0,2020-05-01,NaN,NaN,NaN,NaT,EXEMPTION 4,24,148440.00,148440.00,...,NaN,NaN,NaN,NaN,"BANK OF AMERICA, NATIONAL ASSOCIATION",CHARLOTTE,NC,UNANSWERED,UNANSWERED,NaN
1,1,2020-05-01,LADSON,NaN,29456-3517,NaT,EXEMPTION 4,24,140081.78,140081.78,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),SYNOVUS BANK,COLUMBUS,GA,UNANSWERED,UNANSWERED,NaN
2,2,2020-05-01,MANNING,NaN,29102-2734,2021-04-20,PAID IN FULL,24,136520.96,136520.96,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),SYNOVUS BANK,COLUMBUS,GA,UNANSWERED,UNANSWERED,NaN
3,3,2020-05-01,REDFORD,NaN,48239-2446,2021-04-16,PAID IN FULL,24,126798.00,126798.00,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),"BANK OF AMERICA, NATIONAL ASSOCIATION",CHARLOTTE,NC,MALE OWNED,NON-VETERAN,NaN
4,4,2020-05-01,SPARTANBURG,NaN,29303-2085,NaT,EXEMPTION 4,24,125916.00,125916.00,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),SYNOVUS BANK,COLUMBUS,GA,UNANSWERED,UNANSWERED,NaN


In [25]:
%%time
df[df=='UNANSWERED'].count()

Unnamed: 0                      0
DateApproved                    0
BorrowerCity                    0
BorrowerState                   0
BorrowerZip                     0
LoanStatusDate                  0
LoanStatus                      0
Term                            0
InitialApprovalAmount           0
CurrentApprovalAmount           0
UndisbursedAmount               0
FranchiseName                   0
ServicingLenderName             0
ServicingLenderCity             0
ServicingLenderState            0
RuralUrbanIndicator             0
HubzoneIndicator                0
LMIIndicator                    0
BusinessAgeDescription     531496
ProjectCity                     0
ProjectCountyName               0
ProjectState                    0
CD                              0
JobsReported                    0
NAICSCode                       0
Race                      8257211
Ethnicity                       0
UtilitiesProceed                0
PayrollProceed                  0
MortgageIntPro

In [26]:
%%time
#Replace 'UNANSWERED' with NaN 
df['BusinessAgeDescription'] = df['BusinessAgeDescription'].replace(['UNANSWERED'], np.nan)
df['Race'] = df['Race'].replace(['UNANSWERED'], np.nan)
df['Gender'] = df['Gender'].replace(['UNANSWERED'], np.nan)
df['Veteran'] = df['Veteran'].replace(['UNANSWERED'], np.nan)

CPU times: user 6.27 s, sys: 2.78 s, total: 9.05 s
Wall time: 10.7 s


In [27]:
#Verify that NaN was used
df.head(5)

,Unnamed: 0,DateApproved,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,InitialApprovalAmount,CurrentApprovalAmount,...,RefinanceEidlProceed,HealthcareProceed,DebtIntProceed,BusinessType,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit
0,0,2020-05-01,NaN,NaN,NaN,NaT,EXEMPTION 4,24,148440.00,148440.00,...,NaN,NaN,NaN,NaN,"BANK OF AMERICA, NATIONAL ASSOCIATION",CHARLOTTE,NC,NaN,NaN,NaN
1,1,2020-05-01,LADSON,NaN,29456-3517,NaT,EXEMPTION 4,24,140081.78,140081.78,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),SYNOVUS BANK,COLUMBUS,GA,NaN,NaN,NaN
2,2,2020-05-01,MANNING,NaN,29102-2734,2021-04-20,PAID IN FULL,24,136520.96,136520.96,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),SYNOVUS BANK,COLUMBUS,GA,NaN,NaN,NaN
3,3,2020-05-01,REDFORD,NaN,48239-2446,2021-04-16,PAID IN FULL,24,126798.00,126798.00,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),"BANK OF AMERICA, NATIONAL ASSOCIATION",CHARLOTTE,NC,MALE OWNED,NON-VETERAN,NaN
4,4,2020-05-01,SPARTANBURG,NaN,29303-2085,NaT,EXEMPTION 4,24,125916.00,125916.00,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),SYNOVUS BANK,COLUMBUS,GA,NaN,NaN,NaN


In [28]:
#Replace 'UNKNOWN/NOTSTATED' with NaN 
df['Ethnicity'] = df['Ethnicity'].replace(['UNKNOWN/NOTSTATED'], np.nan)

In [29]:
df['Ethnicity']

0                              NaN
1                              NaN
2                              NaN
3           NOT HISPANIC OR LATINO
4                              NaN
                     ...          
10697214    NOT HISPANIC OR LATINO
10697215                       NaN
10697216                       NaN
10697217                       NaN
10697218                       NaN
Name: Ethnicity, Length: 10697219, dtype: object

In [30]:
#Checking the values of BusinessType
df['BusinessType'].value_counts()

SOLE PROPRIETORSHIP                    3031152
LIMITED  LIABILITY COMPANY(LLC)        2331948
CORPORATION                            2279173
SUBCHAPTER S CORPORATION               1063152
SELF-EMPLOYED INDIVIDUALS               774170
INDEPENDENT CONTRACTORS                 670057
NON-PROFIT ORGANIZATION                 242739
PARTNERSHIP                             117161
LIMITED LIABILITY PARTNERSHIP            58754
SINGLE MEMBER LLC                        46891
PROFESSIONAL ASSOCIATION                 42017
501(C)3 – NON PROFIT                     11687
COOPERATIVE                               9257
501(C)6 – NON PROFIT MEMBERSHIP           5075
NON-PROFIT CHILDCARE CENTER               3920
TRUST                                     2445
JOINT VENTURE                             1610
EMPLOYEE STOCK OWNERSHIP PLAN(ESOP)       1108
QUALIFIED JOINT-VENTURE (SPOUSES)          807
TENANT IN COMMON                           662
HOUSING CO-OP                              571
501(C)19 – NO

In [31]:
#Standarize Nonprofit in Column 'BusinessType'
df['BusinessType'] = df['BusinessType'].replace(to_replace=['NON-PROFIT ORGANIZATION', 
                                                            '501(C)3 – NON PROFIT',
                                                            '501(C)6 – NON PROFIT MEMBERSHIP',
                                                            'NON-PROFIT CHILDCARE CENTER',
                                                            '501(C)19 – NON PROFIT VETERANS',
                                                            '501(C) – NON PROFIT EXCEPT 3,4,6,'], 
                                                value='NON-PROFIT')

In [32]:
#Verify combining NP into one
df['BusinessType'].value_counts()

SOLE PROPRIETORSHIP                    3031152
LIMITED  LIABILITY COMPANY(LLC)        2331948
CORPORATION                            2279173
SUBCHAPTER S CORPORATION               1063152
SELF-EMPLOYED INDIVIDUALS               774170
INDEPENDENT CONTRACTORS                 670057
NON-PROFIT                              263757
PARTNERSHIP                             117161
LIMITED LIABILITY PARTNERSHIP            58754
SINGLE MEMBER LLC                        46891
PROFESSIONAL ASSOCIATION                 42017
COOPERATIVE                               9257
TRUST                                     2445
JOINT VENTURE                             1610
EMPLOYEE STOCK OWNERSHIP PLAN(ESOP)       1108
QUALIFIED JOINT-VENTURE (SPOUSES)          807
TENANT IN COMMON                           662
HOUSING CO-OP                              571
TRIBAL CONCERNS                            118
ROLLOVER AS BUSINESS START-UPS (ROB         38
Name: BusinessType, dtype: int64

In [33]:
#Look at NACIS Values
df['NAICSCode'].value_counts()

812112.0    392624
722511.0    322868
531210.0    238955
812990.0    212663
112111.0    208740
             ...  
314911.0         1
315992.0         1
333294.0         1
514199.0         1
421850.0         1
Name: NAICSCode, Length: 1242, dtype: int64

In [34]:
#Change NAICS Code to string
df['NAICSCode'] = df['NAICSCode'].astype(str)

In [37]:
type(df['NAICSCode'][1])

str

In [35]:
#Verify the string change
df['NAICSCode'].head(5)

0    339114.0
1         nan
2    541990.0
3         nan
4    999990.0
Name: NAICSCode, dtype: object

In [36]:
#Remove the ".0" from NAICS
df['NAICSCode'] = df['NAICSCode'].str.slice(0, -2)

In [36]:
#Verify the slice 
df['NAICSCode']

0           339114
1                n
2           541990
3                n
4           999990
             ...  
10697214    621210
10697215    624410
10697216    238210
10697217    621610
10697218    722511
Name: NAICSCode, Length: 10697219, dtype: object

In [38]:
#Reading NAICS csv file and turning into dict
naics_dict = pd.read_csv('data/2017_NAICS_Codes.csv', header=None, index_col=0, squeeze=True).to_dict()

In [39]:
#Look at the NAICS dict
naics_dict

{'2017 NAICS Code': '2017 NAICS Title',
 nan: nan,
 '111110': 'SOYBEAN FARMING',
 '111120': 'OILSEED (EXCEPT SOYBEAN) FARMING ',
 '111130': 'DRY PEA AND BEAN FARMING ',
 '111140': 'WHEAT FARMING',
 '111150': 'CORN FARMING ',
 '111160': 'RICE FARMING',
 '111191': 'OILSEED AND GRAIN COMBINATION FARMING ',
 '111199': 'ALL OTHER GRAIN FARMING ',
 '111211': 'POTATO FARMING ',
 '111219': 'OTHER VEGETABLE (EXCEPT POTATO) AND MELON FARMING ',
 '111310': 'ORANGE GROVES',
 '111320': 'CITRUS (EXCEPT ORANGE) GROVES ',
 '111331': 'APPLE ORCHARDS ',
 '111332': 'GRAPE VINEYARDS ',
 '111333': 'STRAWBERRY FARMING ',
 '111334': 'BERRY (EXCEPT STRAWBERRY) FARMING ',
 '111335': 'TREE NUT FARMING ',
 '111336': 'FRUIT AND TREE NUT COMBINATION FARMING ',
 '111339': 'OTHER NONCITRUS FRUIT FARMING ',
 '111411': 'MUSHROOM PRODUCTION ',
 '111419': 'OTHER FOOD CROPS GROWN UNDER COVER ',
 '111421': 'NURSERY AND TREE PRODUCTION ',
 '111422': 'FLORICULTURE PRODUCTION ',
 '111910': 'TOBACCO FARMING',
 '111920': 'COTT

In [40]:
%%time
#This takes about 10 min so be aware
#Dict to iterate over column and replace values
for key, value in naics_dict.items():
     df['NAICSCode'] = df['NAICSCode'].replace(key, value)

CPU times: user 10min 7s, sys: 42.4 s, total: 10min 50s
Wall time: 10min 51s


In [41]:
df['NAICSCode']

0                DENTAL EQUIPMENT AND SUPPLIES MANUFACTURING 
1                                                           n
2           ALL OTHER PROFESSIONAL, SCIENTIFIC, AND TECHNI...
3                                                           n
4                                 UNCLASSIFIED ESTABLISHMENTS
                                  ...                        
10697214                                 OFFICES OF DENTISTS 
10697215                             CHILD DAY CARE SERVICES 
10697216    ELECTRICAL CONTRACTORS AND OTHER WIRING INSTAL...
10697217                            HOME HEALTH CARE SERVICES
10697218                            FULL-SERVICE RESTAURANTS 
Name: NAICSCode, Length: 10697219, dtype: object

In [42]:
df['NAICSCode'] = df['NAICSCode'].replace(['n'], np.nan)

In [43]:
df['NAICSCode']

0                DENTAL EQUIPMENT AND SUPPLIES MANUFACTURING 
1                                                         NaN
2           ALL OTHER PROFESSIONAL, SCIENTIFIC, AND TECHNI...
3                                                         NaN
4                                 UNCLASSIFIED ESTABLISHMENTS
                                  ...                        
10697214                                 OFFICES OF DENTISTS 
10697215                             CHILD DAY CARE SERVICES 
10697216    ELECTRICAL CONTRACTORS AND OTHER WIRING INSTAL...
10697217                            HOME HEALTH CARE SERVICES
10697218                            FULL-SERVICE RESTAURANTS 
Name: NAICSCode, Length: 10697219, dtype: object

In [43]:
df.head(5)

,Unnamed: 0,DateApproved,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,InitialApprovalAmount,CurrentApprovalAmount,...,RefinanceEidlProceed,HealthcareProceed,DebtIntProceed,BusinessType,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit
0,0,2020-05-01,NaN,NaN,NaN,NaT,EXEMPTION 4,24,148440.00,148440.00,...,NaN,NaN,NaN,NaN,"BANK OF AMERICA, NATIONAL ASSOCIATION",CHARLOTTE,NC,NaN,NaN,NaN
1,1,2020-05-01,LADSON,NaN,29456-3517,NaT,EXEMPTION 4,24,140081.78,140081.78,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),SYNOVUS BANK,COLUMBUS,GA,NaN,NaN,NaN
2,2,2020-05-01,MANNING,NaN,29102-2734,2021-04-20,PAID IN FULL,24,136520.96,136520.96,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),SYNOVUS BANK,COLUMBUS,GA,NaN,NaN,NaN
3,3,2020-05-01,REDFORD,NaN,48239-2446,2021-04-16,PAID IN FULL,24,126798.00,126798.00,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),"BANK OF AMERICA, NATIONAL ASSOCIATION",CHARLOTTE,NC,MALE OWNED,NON-VETERAN,NaN
4,4,2020-05-01,SPARTANBURG,NaN,29303-2085,NaT,EXEMPTION 4,24,125916.00,125916.00,...,NaN,NaN,NaN,LIMITED LIABILITY COMPANY(LLC),SYNOVUS BANK,COLUMBUS,GA,NaN,NaN,NaN


In [47]:
df["Unnamed: 0"]

0                  0
1                  1
2                  2
3                  3
4                  4
              ...   
10697214    10697214
10697215    10697215
10697216    10697216
10697217    10697217
10697218    10697218
Name: Unnamed: 0, Length: 10697219, dtype: int64

In [50]:
#Drop the additional index
df = df.drop(columns="Unnamed: 0", axis=1)

In [51]:
df.shape

(10697219, 40)

In [52]:
#Output Cleaned File
df.to_csv('cleaned_ppp_data.csv.gz', compression='gzip', index=False)